# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

### import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re
import pickle

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from termcolor import colored

In [21]:
# load data from database
engine = create_engine('sqlite:///response.db')
df = pd.read_sql_table('messages', con=engine)


In [22]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [23]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
#X = df[['message', 'original', 'genre']].values
X = df.message.values
cat_cols = ['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']
Y = df[cat_cols].values 

In [25]:
X.shape

(26216,)

In [26]:
Y.shape

(26216, 36)

In [27]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [28]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 2. Write a tokenization function to process your text data

In [29]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [30]:
word_tokenize(X[0])

['Weather',
 'update',
 '-',
 'a',
 'cold',
 'front',
 'from',
 'Cuba',
 'that',
 'could',
 'pass',
 'over',
 'Haiti']

In [31]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [32]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
pipe = Pipeline([
    ('v', CountVectorizer(tokenizer=tokenize)),
    ('t', TfidfTransformer()),  
    ('cl', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [35]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('v',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fd24b806d30>)),
                ('t', TfidfTransformer()),
                ('cl',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred = pipe.predict(X_test)

In [18]:
y_pred.shape[1]

36

In [19]:
y_pred.shape

(6554, 36)

In [20]:
score = []
for i, col in enumerate(cat_cols):    
    precision, recall, fscore, support = precision_recall_fscore_support(y_test[:, i], y_pred[:, i], average='weighted')
    score.append(fscore)
    print("f-score for category {} is {}".format(colored(col, 'red'), colored(fscore, 'blue')))
    
#average of all f1 scores
print("{}".format(colored('================================================', 'green')))
print("average f-score for all columns is {}".format(colored(sum(score)/len(score), 'red', attrs=['bold'])))

f-score for category related is 0.7662343393347271
f-score for category request is 0.8784743774005084
f-score for category offer is 0.9940533776362491
f-score for category aid_related is 0.7626439432198321
f-score for category medical_help is 0.8926096317037598
f-score for category medical_products is 0.9331840136843472
f-score for category search_and_rescue is 0.9570673650191511
f-score for category security is 0.9747389345794563
f-score for category military is 0.9518347897654964
f-score for category child_alone is 1.0
f-score for category water is 0.9406304303080795
f-score for category food is 0.9223704597933039
f-score for category shelter is 0.9073254855814316
f-score for category clothing is 0.9789390073672429
f-score for category money is 0.9691634683955471
f-score for category missing_people is 0.9807376331924117
f-score for category refugees is 0.9520602816588087
f-score for category death is 0.9403540355425853
f-score for category other_aid is 0.8117033007252119
f-score for 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

In [23]:
pickle.dump(pipe, open('model.pkl', 'wb'))

### 6. Improve your model
Use grid search to find better parameters. 

In [27]:
pipe.get_params()

{'memory': None,
 'steps': [('v',
   CountVectorizer(tokenizer=<function tokenize at 0x7f9a6042c5e0>)),
  ('t', TfidfTransformer()),
  ('cl', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'v': CountVectorizer(tokenizer=<function tokenize at 0x7f9a6042c5e0>),
 't': TfidfTransformer(),
 'cl': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'v__analyzer': 'word',
 'v__binary': False,
 'v__decode_error': 'strict',
 'v__dtype': numpy.int64,
 'v__encoding': 'utf-8',
 'v__input': 'content',
 'v__lowercase': True,
 'v__max_df': 1.0,
 'v__max_features': None,
 'v__min_df': 1,
 'v__ngram_range': (1, 1),
 'v__preprocessor': None,
 'v__stop_words': None,
 'v__strip_accents': None,
 'v__token_pattern': '(?u)\\b\\w\\w+\\b',
 'v__tokenizer': <function __main__.tokenize(text)>,
 'v__vocabulary': None,
 't__norm': 'l2',
 't__smooth_idf': True,
 't__sublinear_tf': False,
 't__use_idf': True,
 'cl__estimator__bootstrap': True,
 'cl__estimator__ccp_alpha': 0.

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
score = []
for i, col in enumerate(category_names):
    report = classification_report(y_test[:,i], y_pred[:, i], zero_division=0, output_dict=True)    
    #print(report['0']['f1-score'])
    score.append(report['0']['f1-score'])
    
#average of all f1 scores
sum(score)/len(score)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.